<a href="https://colab.research.google.com/github/tanisha0016/Sampling-Assignment/blob/main/Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [14]:

url = "https://raw.githubusercontent.com/AnjulaMehto/Sampling_Assignment/main/Creditcard_data.csv"
df = pd.read_csv(url)

In [15]:
df['Class'].value_counts()

,count
Class,
0,763
1,9


In [16]:
X = df.drop(columns=['Class'])
y = df['Class']

Oversampling

In [17]:
smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

In [18]:
df_balanced = pd.DataFrame(X_smote, columns=X.columns)
df_balanced['Class'] = y_smote

In [19]:
df_balanced['Class'].value_counts()

,count
Class,
0,763
1,763


Undersampling

In [20]:
rus = RandomUnderSampler(random_state=42)
X_rus, y_rus = rus.fit_resample(X, y)

Sample Size

In [21]:
def calculate_sample_size(z, p, E, factor=1):
    return int((z**2 * p * (1 - p)) / (E / factor)**2)

In [22]:
z=1.96
p =0.5
E =0.05
S =3
C =4

In [23]:
n_simple = calculate_sample_size(z, p, E)
n_stratified = calculate_sample_size(z, p, E, S)
n_cluster = calculate_sample_size(z, p, E, C)

Sampling Techniques

In [27]:
sample_simple = df_balanced.sample(n_simple, random_state=42)

sample_stratified, _ = train_test_split(df_balanced, test_size=0.8, stratify=df_balanced['Class'], random_state=42)

clusters = df_balanced.groupby('Class', group_keys=False)
cluster_sample_size = min(n_cluster, len(df_balanced))
sample_cluster = clusters.apply(lambda x: x.sample(min(len(x), max(1, int(0.2 * len(x)))), random_state=42))

if len(sample_cluster) > cluster_sample_size:
    sample_cluster = sample_cluster.sample(n=cluster_sample_size, random_state=42)

step = len(df_balanced) // n_simple
sample_systematic = df_balanced.iloc[::step]

<ipython-input-27-356b951420ac>:7: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sample_cluster = clusters.apply(lambda x: x.sample(min(len(x), max(1, int(0.2 * len(x)))), random_state=42))


In [28]:
models = {
    "M1": RandomForestClassifier(random_state=42),
    "M2": SVC(),
    "M3": LogisticRegression(),
    "M4": GaussianNB(),
    "M5": DecisionTreeClassifier()
}

samples = {
    "Sampling1": sample_simple,
    "Sampling2": sample_stratified,
    "Sampling3": sample_cluster,
    "Sampling4": sample_systematic
}

In [29]:
accuracy_results = {}
for model_name, model in models.items():
    accuracy_results[model_name] = {}
    for sample_name, sample_data in samples.items():
        X_sample = sample_data.drop(columns=['Class'])
        y_sample = sample_data['Class']

        X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        accuracy_results[model_name][sample_name] = accuracy

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [32]:

results_df = pd.DataFrame(accuracy_results)

results_df = results_df.T
print(results_df)
results_df.to_csv("results.csv")


    Sampling1  Sampling2  Sampling3  Sampling4
M1   0.974026   0.983607   0.950820   1.000000
M2   0.623377   0.721311   0.754098   0.735294
M3   0.844156   0.901639   0.950820   0.872549
M4   0.857143   0.901639   0.901639   0.882353
M5   0.961039   0.934426   0.885246   0.980392
